# Annotating Female Abolitionist Poets using SpaCy

This notebook will cover the proccess of creating a .csv file with annotated data for the Female Abolitionist Poems dataset using SpaCy (see [README](https://github.com/tijn-donners/collecting_data_spaCy/blob/main/README.md) for more info). First, let's import the required Python libraries


In [5]:
import os #for loading (meta)data
import pandas as pd
import spacy
#!spacy download en_core_web_sm #downloads english language model


### Now that the required modules are imported, we make two lists: filenames and a list of texts

In [6]:
texts = []
file_names = []

for file_name in os.listdir('data'):
    if file_name.endswith('.txt'):
        texts.append(open('data' + '/' + file_name, 'r', encoding='utf-8').read())  #the 'r' is for 'read mode'
        file_names.append(file_name)

### Now we turn these lists into a dictionary, and then a pandas DataFrame

In [7]:
d = {'Filename':file_names,'Text':texts}
df = pd.DataFrame(d)
df.head()

,Filename,Text
0,More_1788.txt,"S L A V E R Y,\nA P O E M.\n\nBY\n\nH A N N A..."
1,Yearsly_1788.txt,A\nP O E M\nON THE\nI N H U M A N I T Y\nOF TH...
2,Barbauld_1791.txt,E P I S T L E\n\nT O\nWILLIAM WILBERFORC...
3,Card_1792_1.txt,A\n\nP O E M\n\nO N T H E\n\nA F R I C...
4,More_1797.txt,CHEAP REPOSITORY\nT H E\nS O R R O W S O F ...


### As we can see, there are a lot of tabs (\t) and newlines (\n) that have to be removed from the raw text

In [9]:
df['Text'] = df['Text'].str.replace(r'\s+', ' ', regex=True).str.strip()
df.head(20)

,Filename,Text
0,More_1788.txt,"S L A V E R Y, A P O E M. BY H A N N A H M O R..."
1,Yearsly_1788.txt,A P O E M ON THE I N H U M A N I T Y OF THE S ...
2,Barbauld_1791.txt,"E P I S T L E T O WILLIAM WILBERFORCE, ESQ. [P..."
3,Card_1792_1.txt,A P O E M O N T H E A F R I C A N S L A V E T ...
4,More_1797.txt,CHEAP REPOSITORY T H E S O R R O W S O F Y A M...
5,Falconar_1788.txt,P O E M S O N S L A V E R Y: B Y MARIA FALCONA...
6,Card_1792_2.txt,A P O E M O N T H E A F R I C A N S L A V E T ...
